# Question Answering – Évaluation du modèle

Ce notebook évalue les performances du modèle fine-tuné
sur le dataset SQuAD (Exact Match, F1-score et temps d’inférence).


## Objectifs

- Charger le modèle fine-tuné
- Évaluer les performances sur le jeu de validation
- Calculer les métriques Exact Match et F1
- Mesurer le temps d’inférence


In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import numpy as np
import time
import torch
import matplotlib.pyplot as plt

c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Paramètres d'évaluation

**Contexte :**
- L'évaluation est effectuée sur un sous-ensemble de 500 exemples afin de limiter le temps de calcul tout en conservant une estimation représentative des performances.
- Pour des raisons de temps de calcul, l'évaluation est réalisée sur une seule fenêtre de contexte (sans sliding window), ce qui peut légèrement sous-estimer les performances réelles.
- Seules les métriques **Exact Match (EM)** et **F1** sont utilisées, conformément à la norme d'évaluation SQuAD.

In [ ]:
# Load original SQuAD data for evaluation (answers are not in tokenized_datasets)
from datasets import load_dataset
raw_datasets = load_dataset("squad")

## Chargement du modèle fine-tuné


In [3]:
model_path = "outputs/checkpoints/roberta/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

model.eval()


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 330.51it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              


RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

## Métriques SQuAD


In [5]:
metric = evaluate.load("squad")


## Fonction d’inférence


In [6]:
def predict_with_score(example):
    inputs = {
        "input_ids": torch.tensor(example["input_ids"]).unsqueeze(0).to(device),
        "attention_mask": torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)
    }

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits.squeeze()
    end_logits = outputs.end_logits.squeeze()

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    score = start_logits[start_idx] + end_logits[end_idx]

    return start_idx.item(), end_idx.item(), score.item()


## Évaluation sur le jeu de validation


In [7]:
n_samples = 500
validation_set = raw_datasets["validation"].select(range(min(n_samples, len(raw_datasets["validation"]))))


In [ ]:
predictions = []
references = []
y_true = []
y_scores = []

for example in validation_set:
    # Tokenize the raw example
    encoded = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=384,
        return_tensors="pt"
    )
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**{k: v.to(device) for k, v in encoded.items()})
    
    start_logits = outputs.start_logits[0]
    end_logits = outputs.end_logits[0]
    
    start_pred = torch.argmax(start_logits).item()
    end_pred = torch.argmax(end_logits).item()
    
    # Fix invalid span
    if end_pred < start_pred:
        end_pred = start_pred
    
    score = start_logits[start_pred] + end_logits[end_pred]

    # Decode prediction
    prediction_text = tokenizer.decode(
        encoded["input_ids"][0][start_pred:end_pred + 1],
        skip_special_tokens=True
    )

    gold_text = example["answers"]["text"][0]

    # Exact Match → label binaire
    y_true.append(int(prediction_text.strip() == gold_text.strip()))
    y_scores.append(score.item())

    # Append to predictions and references for metric computation
    predictions.append({
        "id": example["id"],
        "prediction_text": prediction_text
    })
    
    references.append({
        "id": example["id"],
        "answers": example["answers"]
    })

## Résultats


In [ ]:
results = metric.compute(predictions=predictions, references=references)
results

{'exact_match': 0.2, 'f1': 3.642459670237826}

## Mesure du temps d’inférence


In [ ]:
start_time = time.time()

for example in validation_set:
    encoded = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=384,
        return_tensors="pt"
    )
    
    with torch.no_grad():
        outputs = model(**{k: v.to(device) for k, v in encoded.items()})

if torch.cuda.is_available():
    torch.cuda.synchronize()

end_time = time.time()

avg_time = (end_time - start_time) / n_samples
avg_time

0.03647572088241577

## Conclusion

Le modèle fine-tuné RoBERTa-base a été évalué sur le jeu de validation SQuAD.

Les métriques utilisées sont :
- **Exact Match (EM)** : pourcentage de prédictions identiques aux réponses gold
- **F1** : moyenne harmonique de la précision et du rappel au niveau des tokens
- **Temps d'inférence** : temps moyen pour traiter un exemple

Ces deux métriques (EM et F1) constituent la norme standard d'évaluation SQuAD et sont les plus pertinentes pour cette tâche de question-répondage.

In [ ]:
results_summary = {
    "model": "RoBERTa-base",
    "EM": results["exact_match"],
    "F1": results["f1"],
    "Inference_time_ms": avg_time * 1000
}

results_summary

{'model': 'RoBERTa-base',
 'EM': 0.2,
 'F1': 3.642459670237826,
 'Precision': np.float64(0.003658063808334542),
 'Recall': np.float64(0.5580448065173116),
 'AUC': 0.5571142284569137,
 'Inference_time_ms': 36.47572088241577}

In [14]:
import json

# Sauvegarder les résultats en JSON
with open("outputs/results_roberta.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("Résultats sauvegardés dans outputs/results_roberta.json")


Résultats sauvegardés dans outputs/results_roberta.json


## Résumé des résultats
